In [ ]:
import sys
import torch
import torch.nn.functional as F
from torchvision import datasets,transforms, models
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from natsort import natsorted
from PIL import Image
import os
from torchvision.models import vgg16
from torchvision.models.vgg import VGG
from sklearn.metrics import jaccard_score as jsc

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
main_dir = './image_2/'
label_dir = './semantic/'

pretrained_model = models.vgg16(pretrained = True)
features = list(pretrained_model.features.children())
features[0].padding=(100,100)


class FCN16s(nn.Module):

    def __init__(self, pretrained_net, n_class):
        super().__init__()
        self.n_class = n_class
        self.pretrained_net = pretrained_net
        self.relu    = nn.ReLU(inplace=True)
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(512)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(256)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(128)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(64)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn5     = nn.BatchNorm2d(32)
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        output = self.pretrained_net(x)
        x5 = output['x5']  # size=(N, 512, x.H/32, x.W/32)
        x4 = output['x4']  # size=(N, 512, x.H/16, x.W/16)

        score = self.relu(self.deconv1(x5))               # size=(N, 512, x.H/16, x.W/16)
        score = self.bn1(score + x4)                      # element-wise add, size=(N, 512, x.H/16, x.W/16)
        score = self.bn2(self.relu(self.deconv2(score)))  # size=(N, 256, x.H/8, x.W/8)
        score = self.bn3(self.relu(self.deconv3(score)))  # size=(N, 128, x.H/4, x.W/4)
        score = self.bn4(self.relu(self.deconv4(score)))  # size=(N, 64, x.H/2, x.W/2)
        score = self.bn5(self.relu(self.deconv5(score)))  # size=(N, 32, x.H, x.W)
        score = self.classifier(score)                    # size=(N, n_class, x.H/1, x.W/1)

        return score  # size=(N, n_class, x.H/1, x.W/1)

class VGGNet(VGG):
    def __init__(self, pretrained=True, model='vgg16', requires_grad=True, remove_fc=True, show_params=False):
        super().__init__(make_layers(cfg[model]))
        self.ranges = ranges[model]

        if pretrained:
            exec("self.load_state_dict(models.%s(pretrained=True).state_dict())" % model)

        if not requires_grad:
            for param in super().parameters():
                param.requires_grad = False

        if remove_fc:  # delete redundant fully-connected layer params, can save memory
            del self.classifier

        if show_params:
            for name, param in self.named_parameters():
                print(name, param.size())

    def forward(self, x):
        output = {}

        # get the output of each maxpooling layer (5 maxpool in VGG net)
        for idx in range(len(self.ranges)):
            for layer in range(self.ranges[idx][0], self.ranges[idx][1]):
                x = self.features[layer](x)
            output["x%d"%(idx+1)] = x

        return output

ranges = {
    'vgg11': ((0, 3), (3, 6),  (6, 11),  (11, 16), (16, 21)),
    'vgg13': ((0, 5), (5, 10), (10, 15), (15, 20), (20, 25)),
    'vgg16': ((0, 5), (5, 10), (10, 17), (17, 24), (24, 31)),
    'vgg19': ((0, 5), (5, 10), (10, 19), (19, 28), (28, 37))
}

# cropped version from https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
cfg = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)

vgg_model = VGGNet(requires_grad=False)
model = FCN16s(pretrained_net=vgg_model, n_class=35).to(device)
input = torch.autograd.Variable(torch.randn(1, 3, 256, 1024)).to(device)
output = model(input)
assert output.size() == torch.Size([1, 35, 256, 1024])

class CustomDataset(Dataset):
    def __init__(self, main_dir, label_dir, transform):
        self.main_dir = main_dir
        self.label_dir = label_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        all_imgs_label = os.listdir(label_dir)
        self.total_imgs = natsorted(all_imgs)
        self.total_imgs_label = natsorted(all_imgs_label)
        
    def __len__(self):
        return len(self.total_imgs)
    
    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_loc_label = os.path.join(self.label_dir, self.total_imgs_label[idx] )
        image = Image.open(img_loc).convert('RGB')
        labels = Image.open(img_loc_label).convert('P')
        tensor_image = self.transform(image)
        tensor_label = self.transform(labels)
        return (tensor_image, (tensor_label*255).type(torch.LongTensor))

transform = transforms.Compose([transforms.Resize((256,1024)),
                                transforms.ToTensor()])

train_data = CustomDataset(main_dir, label_dir, transform)

length_train=int(0.7*len(train_data))
length_val= int(0.15*len(train_data))
length_test = len(train_data) - length_train - length_val

train, val, test = torch.utils.data.random_split(train_data, [length_train, length_val, length_test])


training_loader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=False)
validation_loader = torch.utils.data.DataLoader(val, batch_size = 1, shuffle=False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 1, shuffle = False)

for image, labels in training_loader:
  print(image.size(), labels.size())
  break

# model = FCN16s().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001) # fine tuned the lr
micro = []
macro = []
micro_train = []
macro_train = []
epochs = 100
running_loss_history = []
val_running_loss_history = []

for e in range(epochs):
    predlist=torch.zeros(0,dtype=torch.long, device='cpu')
    labellist=torch.zeros(0,dtype=torch.long, device='cpu')

    predlist_val=torch.zeros(0,dtype=torch.long, device='cpu')
    labellist_val=torch.zeros(0,dtype=torch.long, device='cpu')
    running_loss = 0.0
    val_running_loss = 0.0
    
    
    for inputs, labels in training_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels[:,0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _,preds = torch.max(outputs, dim = 1)
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        labellist=torch.cat([labellist,labels.view(-1).cpu()])
   

    else:
      with torch.no_grad(): # we do not need gradient for validation.
        for val_inputs, val_labels in validation_loader:
          val_inputs = val_inputs.to(device)
          val_labels = val_labels.to(device)
          val_outputs = model(val_inputs)
          val_loss = criterion(val_outputs, val_labels[:,0])
          
          _, val_preds = torch.max(val_outputs, 1)
          val_running_loss += val_loss.item()

          predlist_val=torch.cat([predlist_val,val_preds.view(-1).cpu()])
          labellist_val=torch.cat([labellist_val,val_labels.view(-1).cpu()])

    lbl = predlist.cpu().numpy().reshape(-1)
    target = labellist.cpu().numpy().reshape(-1)
    iou_micro = jsc(target, lbl, average = 'micro' )
    iou_macro = jsc(target, lbl, average = 'macro' )
    micro_train.append(iou_micro)
    macro_train.append(iou_macro)
    epoch_loss = running_loss/len(training_loader)
    running_loss_history.append(epoch_loss)
    val_epoch_loss = val_running_loss/len(validation_loader)
    val_running_loss_history.append(val_epoch_loss)


    lbl_val = predlist_val.cpu().numpy().reshape(-1)
    target_val = labellist_val.cpu().numpy().reshape(-1)
    iou_val_micro = jsc(target_val, lbl_val, average = 'micro' )
    iou_val_macro = jsc(target_val, lbl_val, average = 'macro' )
    micro.append(iou_val_micro)
    macro.append(iou_val_macro)
    print('epoch :', (e+1))       
    print('training loss: {:.4f}, training iou(macro): {:.4f}, training iou(micro): {:.4f}'.format(epoch_loss, iou_macro, iou_micro))
    print('validation loss: {:.4f}, validation iou(macro): {:.4f}, validation iou(micro): {:.4f}'.format(val_epoch_loss, iou_val_macro, iou_val_micro))

plt.figure()
plt.style.use('ggplot')
plt.plot(running_loss_history, label='training loss')
plt.plot(val_running_loss_history, label='validation loss')
plt.legend()
plt.savefig('./Loss.png', dpi=256)

plt.figure()
plt.style.use('ggplot')
#plt.plot(micro, label=' loss')
plt.plot(micro, label='IOU Val')
#plt.plot(micro, label=' loss')
plt.plot(micro_train, label='IOU Train')
plt.legend()
plt.savefig('./IOU_micro.png', dpi=256)

plt.figure()
plt.style.use('ggplot')
#plt.plot(micro, label=' loss')
plt.plot(macro, label='IOU Validation')
#plt.plot(micro, label=' loss')
plt.plot(macro_train, label='IOU Training')
plt.legend()
plt.savefig('./IOU_macro.png', dpi=256)

test_running_corrects = 0.0
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
labellist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad(): # we do not need gradient for validation.
      for test_inputs, test_labels in test_loader:
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        test_outputs = model(test_inputs)
        _, test_preds = torch.max(test_outputs, 1)
        predlist=torch.cat([predlist,test_preds.view(-1).cpu()])
        labellist=torch.cat([labellist,test_labels.view(-1).cpu()])
        test_running_corrects += torch.sum(test_preds == test_labels[:,0].data)


test_acc = test_running_corrects.float()/ (len(test_loader)*1024*256)
print('test acc {:.4f} '.format(test_acc))

from sklearn.metrics import jaccard_score as jsc

lbl = predlist.cpu().numpy().reshape(-1)
target = labellist.cpu().numpy().reshape(-1)
print(jsc(target, lbl, average = 'micro' ))

final_data = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False)
test_running_corrects = 0.0
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
labellist=torch.zeros(0,dtype=torch.long, device='cpu')
i = 0
j = 180    #this image 
with torch.no_grad(): # we do not need gradient for validation.
      for test_inputs, test_labels in final_data:
        if i == j:
          test_inputs = test_inputs.to(device)
          test_labels = test_labels.to(device)
          test_outputs = model(test_inputs)
          _, test_preds = torch.max(test_outputs, 1)
          predlist=torch.cat([predlist,test_preds.view(-1).cpu()])
          labellist=torch.cat([labellist,test_labels.view(-1).cpu()])
          test_running_corrects += torch.sum(test_preds == test_labels[:,0].data)
          break
        else:
          i = i+1
        

from sklearn.metrics import jaccard_score as jsc

lbl = predlist.cpu().numpy().reshape(-1)
target = labellist.cpu().numpy().reshape(-1)
print(jsc(target, lbl, average = 'micro' ))


final_data = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=False)
test_running_corrects = 0.0
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
labellist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad(): # we do not need gradient for validation.
      for test_inputs, test_labels in final_data:
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        test_outputs = model(test_inputs)
        _, test_preds = torch.max(test_outputs, 1)
        predlist=torch.cat([predlist,test_preds.view(-1).cpu()])
        labellist=torch.cat([labellist,test_labels.view(-1).cpu()])
        test_running_corrects += torch.sum(test_preds == test_labels[:,0].data)
        break

test_acc = test_running_corrects.float()/(1024*256)
print('test acc {:.4f} '.format(test_acc))

from collections import  defaultdict
d = defaultdict(set,
            {-1: (0, 0, 142),
             0: (0, 0, 0),
             1: (0, 0, 0),
             2: (0, 0, 0),
             3: (0, 0, 0),
             4: (0, 0, 0),
             5: (111, 74, 0),
             6: (81, 0, 81),
             7: (128, 64, 128),
             8: (244, 35, 232),
             9: (250, 170, 160),
             10: (230, 150, 140),
             11: (70, 70, 70),
             12: (102, 102, 156),
             13: (190, 153, 153),
             14: (180, 165, 180),
             15: (150, 100, 100),
             16: (150, 120, 90),
             17: (153, 153, 153),
             18: (153, 153, 153),
             19: (250, 170, 30),
             20: (220, 220, 0),
             21: (107, 142, 35),
             22: (152, 251, 152),
             23: (70, 130, 180),
             24: (220, 20, 60),
             25: (255, 0, 0),
             26: (0, 0, 142),
             27: (0, 0, 70),
             28: (0, 60, 100),
             29: (0, 0, 90),
             30: (0, 0, 110),
             31: (0, 80, 100),
             32: (0, 0, 230),
             33: (119, 11, 32)})

for img,lab in training_loader:
  print(img.size())
  break

d

test_preds.size()

predictions = test_preds.cpu().numpy()
predictions.shape

lst = []
p = predictions[0,:]
for i in range(len(p)):
  for j in range(len(p[i])):
    lst.append((d[p[i][j]]))
arr = np.array(lst)

arr.shape
arr = np.reshape(arr,[256,1024,3])

p
arr

g = np.zeros([256,1024,3])
for i in range(len(arr)):
  for j in range(len(arr[i])):
    for k in range(len(arr[i][j])):
      g[i][j][k] = arr[i][j][k]/255.0

import cv2
cv2.imwrite('color_img.jpg',arr)
c = cv2.imread('color_img.jpg', 1)
c = cv2.cvtColor(c, cv2.COLOR_BGR2RGB)
cv2.imwrite('Final_img.jpg', c)

torch.save(model, "./entire_model_fcn32_final.pt")
# model = torch.load('./drive/My Drive/hw5/entire_model_fcn32_final.pt')
# model.eval()
# t = test_preds.cpu().numpy()
